# Training an NVIDIA HugeCTR model with Vertex AI Training



In [1]:
import json
import os
import time
import nvtabular as nvt
import shutil

from nvtabular.columns.schema import ColumnSchema, Schema
from nvtabular.tags import Tags
from google.cloud import aiplatform

## Configure environment settings


In [2]:
PROJECT = 'jk-mlops-dev'
REGION = 'us-central1'
GCS_BUCKET = 'gs://jk-vertex-merlin'
LOCAL_STAGING_PATH = '/home/jupyter/staging'
VERTEX_SA = 'vertex-sa@jk-mlops-dev.iam.gserviceaccount.com'

In [3]:
if os.path.isdir(LOCAL_STAGING_PATH):
    shutil.rmtree(LOCAL_STAGING_PATH)
os.makedirs(LOCAL_STAGING_PATH)

### Initialize Vertex AI SDK

In [4]:
aiplatform.init(
    project=PROJECT,
    location=REGION,
    staging_bucket=GCS_BUCKET
)

## Prepare a custom training container

In [5]:
IMAGE_NAME = 'hugectr_deepfm'
IMAGE_URI = f'gcr.io/{PROJECT}/{IMAGE_NAME}'
DOCKERFILE = 'src/training/hugectr'

In [6]:
#! gcloud builds submit --tag {IMAGE_URI} {DOCKERFILE}

In [7]:
!docker build -t {IMAGE_URI} {DOCKERFILE}

Sending build context to Docker daemon  83.97kB
Step 1/4 : FROM nvcr.io/nvidia/merlin/merlin-training:21.09
 ---> 8f6ef763d770
Step 2/4 : RUN pip3 install cloudml-hypertune
 ---> Using cache
 ---> ab2a50ff26f5
Step 3/4 : WORKDIR /src
 ---> Using cache
 ---> ca01d83e287f
Step 4/4 : COPY trainer ./trainer
 ---> Using cache
 ---> d03962dec90c
Successfully built d03962dec90c
Successfully tagged gcr.io/jk-mlops-dev/hugectr_deepfm:latest


In [8]:
!docker push {IMAGE_URI}

Using default tag: latest
The push refers to repository [gcr.io/jk-mlops-dev/hugectr_deepfm]

c107e26d: Preparing 
d791a71c: Preparing 
36e180cd: Preparing 
1f36f93d: Preparing 
e0fa48ac: Preparing 
d4cd8fbb: Preparing 
2cf6a026: Preparing 
26a95f43: Preparing 
ba5d6668: Preparing 
a9641b18: Preparing 
42012c23: Preparing 
bd3e801f: Preparing 
7039f00c: Preparing 
023e019c: Preparing 
f9e4afbc: Preparing 
e9115178: Preparing 
220871e3: Preparing 
76810591: Preparing 
7b7db32e: Preparing 
176606fb: Preparing 
44765ddc: Preparing 
295911d7: Preparing 
1a643992: Preparing 
6f8bf009: Preparing 
d3ad3992: Preparing 
d0bdb6b2: Preparing 
6a95f43: Waiting g 
9b0b029e: Preparing 
3c6245a7: Preparing 
9f6effa5: Preparing 
a5d6668: Waiting g 
31c5510f: Preparing 
9641b18: Waiting g 
e3d1aa10: Preparing 
4cd8fbb: Waiting g 
2012c23: Waiting g 
58a72969: Preparing 
cb179a54: Preparing 
d3e801f: Waiting g 
23e019c: Waiting g 
deda2561: Preparing 
7e454744: Preparing 
039f00c: Waiting g 
9e4afbc: Wa

## Configure a training job

### Set paths to training and validation datasets

In [9]:
DATA_ROOT = 'gs://jk-criteo-bucket/criteo_processed_parquet'
TRAIN_DATA = f'{DATA_ROOT}/train/_file_list.txt'
VALID_DATA = f'{DATA_ROOT}/valid/_file_list.txt'
#TRAIN_METADATA = f'{DATA_ROOT}/train/_metadata.json'
#VALID_METADATA = f'{DATA_ROOT}/valid/_metadata.json'
SCHEMA_PATH = f'{DATA_ROOT}/train/schema.pbtxt'

### Retrieve cardinalities for categorical columns

In [10]:
LOCAL_SCHEMA_PATH = f'{LOCAL_STAGING_PATH}/schema.pbtxt'

!gsutil cp {SCHEMA_PATH} {LOCAL_SCHEMA_PATH}

Copying gs://jk-criteo-bucket/criteo_processed_parquet/train/schema.pbtxt...
/ [1 files][ 20.8 KiB/ 20.8 KiB]                                                
Operation completed over 1 objects/20.8 KiB.                                     


In [11]:
schema = Schema.load_protobuf(LOCAL_SCHEMA_PATH)

In [12]:
def retrieve_cardinalities(schema):
    cardinalities = {key: value.properties['embedding_sizes']['cardinality'] 
                     for key, value in schema.column_schemas.items()
                     if Tags.CATEGORICAL in value.tags}
    
    return cardinalities
    
    
cardinalities = retrieve_cardinalities(schema)
cardinalities

{'C1': 18792578.0,
 'C2': 35176.0,
 'C3': 17091.0,
 'C4': 7383.0,
 'C5': 20154.0,
 'C6': 4.0,
 'C7': 7075.0,
 'C8': 1403.0,
 'C9': 63.0,
 'C10': 12687136.0,
 'C11': 1054830.0,
 'C12': 297377.0,
 'C13': 11.0,
 'C14': 2209.0,
 'C15': 10933.0,
 'C16': 113.0,
 'C17': 4.0,
 'C18': 972.0,
 'C19': 15.0,
 'C20': 19550853.0,
 'C21': 5602712.0,
 'C22': 16779972.0,
 'C23': 375290.0,
 'C24': 12292.0,
 'C25': 101.0,
 'C26': 35.0}

### Configure worker pools

#### Training module

In [13]:
TRAINING_MODULE = 'trainer.task'

#### HugeCTR model and trainer configuration

In [14]:
NUM_EPOCHS = 0
MAX_ITERATIONS = 10000
EVAL_INTERVAL = 1000
EVAL_BATCHES = 500
EVAL_BATCHES_FINAL = 5000
DISPLAY_INTERVAL = 200
SNAPSHOT_INTERVAL = 0
WORKSPACE_SIZE_PER_GPU = 61
PER_GPU_BATCHSIZE = 2048
LR = 0.001
DROPOUT_RATE = 0.5
NUM_WORKERS = 12
SLOT_SIZE_ARRAY = json.dumps(
    [int(cardinality) for cardinality in cardinalities.values()]).replace(' ', '')

#### Training node configuration

In [15]:
MACHINE_TYPE = 'a2-highgpu-4g'
ACCELERATOR_TYPE = 'NVIDIA_TESLA_A100'
ACCELERATOR_NUM = 4

#### Worker pool specification

In [16]:
job_name = 'HUGECTR_{}'.format(time.strftime("%Y%m%d_%H%M%S"))
batchsize = PER_GPU_BATCHSIZE * ACCELERATOR_NUM
gpus = json.dumps([list(range(ACCELERATOR_NUM))]).replace(' ','')
model_dir = f'{GCS_BUCKET}/job_dir/{job_name}'
                 
worker_pool_specs =  [
    {
        "machine_spec": {
            "machine_type": MACHINE_TYPE,
            "accelerator_type": ACCELERATOR_TYPE,
            "accelerator_count": ACCELERATOR_NUM,
        },
        "replica_count": 1,
        "container_spec": {
            "image_uri": IMAGE_URI,
            "command": ["python", "-m", TRAINING_MODULE],
            "args": [
                '--batchsize=' + str(batchsize),
                '--train_data=' + TRAIN_DATA.replace('gs://', '/gcs/'), 
                '--valid_data=' + VALID_DATA.replace('gs://', '/gcs/'),
                '--slot_size_array=' + SLOT_SIZE_ARRAY,
                '--max_iter=' + str(MAX_ITERATIONS),
                '--max_eval_batches=' + str(EVAL_BATCHES),
                '--eval_batches=' + str(EVAL_BATCHES_FINAL),
                '--dropout_rate=' + str(DROPOUT_RATE),
                '--num_workers=' + str(NUM_WORKERS),
                '--num_epochs=' + str(NUM_EPOCHS),
                '--eval_interval=' + str(EVAL_INTERVAL),
                '--snapshot=' + str(SNAPSHOT_INTERVAL),
                '--display_interval=' + str(DISPLAY_INTERVAL),
                '--workspace_size_per_gpu=' + str(WORKSPACE_SIZE_PER_GPU),
                '--gpus=' + gpus,
                '--model_dir=' + model_dir
            ],
        },
    }
]

## Submit and monitor a training job

In [ ]:
job = aiplatform.CustomJob(
    display_name=job_name,
    worker_pool_specs=worker_pool_specs,
)
job.run(
    sync=True,
    service_account=VERTEX_SA,
    restart_job_on_worker_restart=False
)

INFO:google.cloud.aiplatform.jobs:Creating CustomJob
INFO:google.cloud.aiplatform.jobs:CustomJob created. Resource name: projects/895222332033/locations/us-central1/customJobs/91965351570440192
INFO:google.cloud.aiplatform.jobs:To use this CustomJob in another session:
INFO:google.cloud.aiplatform.jobs:custom_job = aiplatform.CustomJob.get('projects/895222332033/locations/us-central1/customJobs/91965351570440192')
INFO:google.cloud.aiplatform.jobs:View Custom Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/91965351570440192?project=895222332033
INFO:google.cloud.aiplatform.jobs:CustomJob projects/895222332033/locations/us-central1/customJobs/91965351570440192 current state:
JobState.JOB_STATE_PENDING
INFO:google.cloud.aiplatform.jobs:CustomJob projects/895222332033/locations/us-central1/customJobs/91965351570440192 current state:
JobState.JOB_STATE_PENDING
INFO:google.cloud.aiplatform.jobs:CustomJob projects/895222332033/locations/us-central1/customJobs

## Submit and monitor a hyperparameter tuning job